In [1]:
import numpy as np
import pandas as pd
import re
from os.path import join

from tools import *
from S2.utils._path import *

from sklearn.decomposition import LatentDirichletAllocation
from scipy.spatial import distance

import matplotlib.pyplot as plt
import numpy as np

## rename EXTRACT_FEATURES to DIMENSION_REDUCTION
read_prefix = "/data/MIMIC3/"
write_prefix = "/data/liu/LDA"
res_prefix = "/data/liu/LDA/lda_result"
res_r_prefix = "/data/liu/LDA/lda_R_result/"

In [2]:
sample_epis_file='HADM_ID_SAMPLE_PER_PATIENT'
sampled_hadmid = read_data(join(singledrug_prefix,sample_epis_file),dtype=str)
sampled_hadmid.head()

,SUBJECT_ID,HADM_ID
0,2,163353
1,4,185777
2,6,107064
3,8,159514
4,9,150750


In [3]:
def print_statistics(df):
    
    print("# of rows: %d"%len(df))
    print("# of patients: %d"%len(df['SUBJECT_ID'].unique()))
    print("# of patients: %d"%len(df['HADM_ID'].unique()))

In [4]:
def df2matrix(df,col):
    print("Original Data:")
    print_statistics(df)
    df_sampled = left_join(sampled_hadmid,df,list(sampled_hadmid.columns))
    print("Sampled Data:")
    print_statistics(df_sampled)
    
    # pres_matrix = pres_df_1k_1k_hadm.pivot(index='SUBJECT_ID', columns='NDC', values='VALUE').fillna(0).reset_index()

    return df_sampled

In [19]:
## ICD9_CODE, 1
diag_df = read_data(join(read_prefix,"DIAGNOSES_ICD"),dtype={"SUBJECT_ID":str,"HADM_ID":str})
diag_df.head()
df2matrix(diag_df,"aa")

Original Data:/n
# of rows: 651047
# of patients: 46520
# of patients: 58976
Sampled Data:/n
# of rows: 445514
# of patients: 38763
# of patients: 38763


,SUBJECT_ID,HADM_ID,ROW_ID,SEQ_NUM,ICD9_CODE
0,2,163353,1,1.0,V3001
1,2,163353,2,2.0,V053
2,2,163353,3,3.0,V290
3,4,185777,13,1.0,042
4,4,185777,14,2.0,1363
...,...,...,...,...,...
445509,99999,113369,651043,1.0,75612
445510,99999,113369,651044,2.0,7861
445511,99999,113369,651045,3.0,4019
445512,99999,113369,651046,4.0,25000


In [12]:
pres_df = read_data(
    join(read_prefix,'PRESCRIPTIONS'),
    dtype={"SUBJECT_ID":str,"HADM_ID":str,"NDC":str})[["SUBJECT_ID","HADM_ID","NDC"]].dropna(subset=['NDC'])
pres_df = pres_df[pres_df['NDC']!=0]
# read_data(join(read_prefix,"DIAGNOSES_ICD"),dtype={"SUBJECT_ID":str,"HADM_ID":str})

df2matrix(pres_df,"aa")

Original Data:
# of rows: 4151987
# of patients: 39360
# of patients: 50212
Sampled Data:
# of rows: 3167448
# of patients: 38763
# of patients: 38763


,SUBJECT_ID,HADM_ID,NDC
0,2,163353,63323017302
1,2,163353,0
2,2,163353,63323038810
3,2,163353,0
4,4,185777,0
...,...,...,...
3167443,99999,113369,66993084225
3167444,99999,113369,66993084225
3167445,99999,113369,66993084225
3167446,99999,113369,66993084225


In [5]:
## ITEMID, VALUE
lab_df=read_data(
    join(read_prefix,'LABEVENTS'),
    dtype={"SUBJECT_ID":str,"HADM_ID":str})[["SUBJECT_ID","HADM_ID","ITEMID","VALUE"]].dropna(subset=['VALUE'])
df2matrix(lab_df,"aa")

Original Data:
# of rows: 27852810
# of patients: 46252
# of patients: 58152
Sampled Data:
# of rows: 15412168
# of patients: 38763
# of patients: 38763


,SUBJECT_ID,HADM_ID,ITEMID,VALUE
0,2,163353,51143.0,0
1,2,163353,51144.0,0
2,2,163353,51146.0,0
3,2,163353,51200.0,0
4,2,163353,51221.0,0
...,...,...,...,...
15412163,99999,113369,51250.0,93
15412164,99999,113369,51265.0,193
15412165,99999,113369,51277.0,12.9
15412166,99999,113369,51279.0,3.03


In [6]:
temp_df = df2matrix(lab_df,"")
# [["HADM_ID","ITEMID","VALUE"]].drop_duplicates().pivot(index='HADM_ID', columns='ITEMID', values='VALUE').reset_index()
# matrix.head()


Original Data:
# of rows: 27852810
# of patients: 46252
# of patients: 58152
Sampled Data:
# of rows: 15412168
# of patients: 38763
# of patients: 38763


In [9]:
temp_df.groupby(["HADM_ID","ITEMID"])["VALUE"].mean()

DataError: No numeric types to aggregate

In [14]:
temp_df["real"] = temp_df["VALUE"].apply(lambda s: pd.to_numeric(s, errors='coerce'))
temp_df

,SUBJECT_ID,HADM_ID,ITEMID,VALUE,real
0,2,163353,51143.0,0,0.00
1,2,163353,51144.0,0,0.00
2,2,163353,51146.0,0,0.00
3,2,163353,51200.0,0,0.00
4,2,163353,51221.0,0,0.00
...,...,...,...,...,...
15412163,99999,113369,51250.0,93,93.00
15412164,99999,113369,51265.0,193,193.00
15412165,99999,113369,51277.0,12.9,12.90
15412166,99999,113369,51279.0,3.03,3.03


In [16]:
print(len(temp_df))


15412168


NameError: name 'lenn' is not defined

In [19]:
from S2.cluster_analysis import feature_creation

In [17]:
print(len(temp_df.drop_duplicates()))

10417738


In [ ]:
fc = feature_creation()
tt1 = fc.sampling(temp_df,1,["HADM_ID","ITEMID"],"VALUE")

In [15]:
temp_df[temp_df["real"].isna()]

,SUBJECT_ID,HADM_ID,ITEMID,VALUE,real
18,2,163353,51137.0,1+,NaN
25,2,163353,51233.0,NORMAL,NaN
27,2,163353,51246.0,1+,NaN
32,2,163353,51252.0,NORMAL,NaN
38,2,163353,51266.0,NORMAL,NaN
...,...,...,...,...,...
15411818,99992,197084,51506.0,Clear,NaN
15411819,99992,197084,51508.0,Yellow,NaN
15411820,99992,197084,51514.0,NEG,NaN
15411973,99995,137810,51519.0,NONE,NaN


In [9]:
## procedures
procedure_df=read_data(
    join(read_prefix,'PROCEDURES_ICD'),
    dtype={"SUBJECT_ID":str,"HADM_ID":str})
procedure_df.head()


,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,944,62641,154460,3,3404
1,945,2592,130856,1,9671
2,946,2592,130856,2,3893
3,947,55357,119355,1,9672
4,948,55357,119355,2,331


In [10]:
df2matrix(procedure_df,"aa")

Original Data:
# of rows: 240095
# of patients: 42214
# of patients: 52243
Sampled Data:
# of rows: 174280
# of patients: 38763
# of patients: 38763


,SUBJECT_ID,HADM_ID,ROW_ID,SEQ_NUM,ICD9_CODE
0,2,163353,109376.0,1.0,9955.0
1,4,185777,74266.0,1.0,3893.0
2,4,185777,74267.0,2.0,8872.0
3,4,185777,74268.0,3.0,3323.0
4,6,107064,91635.0,1.0,5569.0
...,...,...,...,...,...
174275,99999,113369,174285.0,1.0,8108.0
174276,99999,113369,174286.0,2.0,8051.0
174277,99999,113369,174287.0,3.0,8162.0
174278,99999,113369,174288.0,4.0,9979.0
